# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Orchestrator - Workers
Examples are based from [Intellectronica - Building Effective Agents with Pydantic AI](https://github.com/intellectronica/building-effective-agents-with-pydantic-ai)

In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
import json
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['azure:gpt-4o', 'azure:gpt-4o-mini']

Using AI model: azure:gpt-4o
Configuring Azure AI Foundry model: gpt-4o at https://agent-workshop-yrkd.cognitiveservices.azure.com/


### Workflow: Orchestrator - Workers

An orchestrator is an agent that determines what work needs to be done
and divides it so that it can be delegated to multiple workers. Once
the work plan is ready, we run each task by a separate worker in parallel
and eventually collect the results from all workers.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> The schemas defined by Pydantic are translated into JSON schema, which is passed on
> to the model API for defining how the result should be structured. Each API handles
> this a bit differentls, but the details are abstracted by the library. In general,
> some complexity and nesting of types is usually OK. In this case, we define a `Task`
> type, and return a list of tasks as part of `OrchestratorResponse`.
>
> Agents in Pydantic AI have some fixed definitions, like their `system_prompt`, and
> can be used to execute multiple runs, each with its own prompt. Here we define a
> worker agent instance, which can execute each task definition.

In [3]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of minion task approach. '
        'For example: "enthusiastic", "technical", "banana-focused", "gadget-oriented", ...'))
    description: str = Field(
        ...,
        description='Clear description for executing this minion task.'
    )


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the minion task and which variations '
        'would be valuable for Gru\'s operations. Focus on how each approach serves '
        'different aspects of minion management or evil schemes.'
    ))
    tasks: List[Task] = Field(..., description="List of minion tasks")


async def orchestrate(task: str) -> Dict:
    """Process minion task by breaking it down and running subtasks in parallel."""

    orchestrator_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'You are Gru\'s master orchestrator for minion operations. ',
            'Analyze this minion-related task and break it down into ',
            '2-3 distinct approaches that different types of minions could handle.',
        ),
        output_type=OrchestratorResponse,
    )
    orchestrator_response = await orchestrator_agent.run(task)
    
    analysis = orchestrator_response.output.analysis
    tasks = orchestrator_response.output.tasks
    
    show('', title='Minion Orchestrator Output')
    show(analysis, title='Analysis')
    show([task.model_dump() for task in tasks], title='Minion Tasks')
    
    # Process all the minion tasks in parallel and collect results
    worker_agent = Agent(
        AI_MODEL,
        system_prompt='You are a specialized minion worker. Generate content based on the minion task specification. Use minion language and enthusiasm where appropriate.',
    )
    worker_responses = await asyncio.gather(*[
        worker_agent.run(json.dumps(
            {'original_minion_task': task} | task_info.model_dump()
        ))
        for task_info in tasks
    ])

    for task, response in zip(tasks, worker_responses):
        show(response.output, title=f"Minion Worker Result ({task.type})")

In [4]:
await orchestrate(
    'Create a comprehensive training manual for new minions joining Gru\'s evil operations, covering banana breaks, gadget safety, and proper "BELLO!" etiquette.'
)

### Minion Orchestrator Output

### Analysis

```
Creating a comprehensive training manual for new minions involves blending fun, practicality, and attention to detail. Different approaches allow leveraging various strengths of minions, enhancing overall efficiency and engagement. Each approach brings a unique flavor to management or schemes, ensuring adaptability and enriched minion skills.
```

### Minion Tasks

```
[{'description': 'Develop a lively and colorful manual filled with '
                 'entertaining illustrations and stories to engage the minions '
                 'in learning. Focus on capturing attention with humor and '
                 "enthusiasm, emphasizing the joy of working in Gru's "
                 'operations. Include playful activities related to banana '
                 'breaks and interactive sections on gadget safety, ensuring '
                 'minions understand the importance in a fun way.',
  'type': 'enthusiastic'},
 {'description': 'Create an informative and structured manual that '
                 'systematically covers all necessary protocols. This approach '
                 'targets more analytical minions, detailing gadget handling '
                 'and safety precautions with clear instructions, diagrams, '
                 'and checklists. Emphasize the technical aspects of '
                 "operations and the logic behind effective 'BELLO!' "
                 'etiquette, with troubleshooting guides for gadgets.',
  'type': 'technical'},
 {'description': 'Design a manual aimed at developing future leaders among '
                 'minions. This would cover the strategic significance of '
                 'banana breaks, advanced gadget management, and leadership '
                 'communication, including setting a proper example with '
                 "'BELLO!' etiquette. Integrate leadership quizzes and "
                 'scenarios to prepare minions for situations requiring quick '
                 'decision-making and teamwork coordination.',
  'type': 'leadership-focused'}]
```

...

# **BELLO! Gru's Great Minion Manual**

### **Welcome, New Minions!**

Bello, bello, bello! Welcome to the wacky and wonderful world of working with the amazing Gru! This guide will help you learn all the ins and outs of being the best minion you can be! Let’s make mischief and glory together, but remember, safety and fun come first. Let's get started, yeeha!

---

## **Chapter 1: Banana Break Bonanza**

### **The Joy of Banana Time:**

🥳 **Banana Break Rule Number One:** Always have FUN! Bananas taste better when shared with a minion friend! Nom-nom-nom!

🔄 **Banana Sharing Circle:** Form a banana-sharing circle every day at noon. Share your banana, share a story. Spread the joy, and don’t forget to say “Tank yu!” to your fellow minions.

🕺 **Banana Dance Boogie:** After eating, celebrate with the Banana Dance! Tap-tap your little feet, and cha-cha-cha like there’s no tomorrow!

**Mischievous Minion Tip:** Be cautious of banana peel slip-slop! Clean up to keep the banana break area neat and tidy.

---

## **Chapter 2: Gadget Safety Giggles**

### **Handling Gru’s Gadgets:**

🔍 **Observe before you go kaboom!** Each gadget has sparkle-sparkle instructions. Follow to stay safe and sound!

🧤 **Get the Right Gear:** Safety goggles make you look cool! Gloves, capes, and whatever super fun gear Gru gives you. Always wear them when tink-tinkering with gadgets.

🔬 **The Test Test:** Every gadget needs a test! Remember: It's gizmo first, minion second. If it goes "BOOM," reset and try again with a chuckle!

📚 **Interactive Safety Game:** "Gadget Guess-a-Rama!" What does this button do? Play with your minion team and guess right to win a super sticker!

**Minion Memory Rhyme:**
"Before going wild, check that switch,
To avoid a boom-boom glitch!"

---

## **Chapter 3: Proper “BELLO!” Etiquette**

### **The Minion Greeting:**

👋 **The Ultimate “BELLO!”:** Always start and end your day with a heartfelt “BELLO!” It makes Gru happy and keeps the evil spirit alive!

🗣️ **Voice Practice Exercise:** Squeeze your cheeks, and say “BELLO!” five times fast! Laugh, giggle, then say it more!

🙋 **The Double Bello Wave:** Greet fellow minions with a double wave. One hand, then the other. Eyes twinkle and repeat “BELLO!” until they grin back.

🥳 **”BELLO!” Bell Ringer:** Practice with the "Bello Bell" in the cafeteria! Hit the bell and make a joyful noise echo with your enthusiastic greeting.

**Enthusiastic Ending:**
"Happiness spreads with a ‘BELLO!’ loud,
Brighten a room, make Gru so proud!”

---

**Congratulations, New Minion!** Now that you've read this manual, go forth and embrace the chaos—with safety, fun, and a whole lot of bananas! Your journey with Gru has just begun. Remember, every gadget, every peel cleaned, every Bello enjoyed makes you a hero in our hilarious, evil mission!

**Minion Motto:**
"Together we yell: BELLO! And rule the day with a cheer!"

### **Bananaramas and BELLO Bumps to All!** 📯🍌🎉

### Minion Worker Result (technical)

# Minion Training Manual

---

## Chapter 1: **BELLO! Etiquette**

### 1.1 Understanding BELLO!
- **BELLO! Basics:** "BELLO!" is our universal greeting! Use it to say hello with enthusiasm and friendliness.  
- **Tone and Volume:** Always sounds cheerful! Not too loud, not too soft, juuust right, like in "BAANAAANAA!"  
- **Situational Awareness:** When meeting new minions or bosses, a polite "BELLO!" can lead to more "BANANA!" rewards!

### 1.2 BELLO! Troubleshooting
- **Not Responded:** Try again with more enthusiasm!
- **Misunderstood:** Combine gestures to aid communication. Maybe add a smile or a little dance!

---

## Chapter 2: **Banana Break Protocol**

### 2.1 Timing Yon Banana Break
- **Duration:** Always take a break for 10 minutes after every 2 hours of hard work. Don't forget to share!
- **Location:** Banana Zone is your safe spot for oh-so-sweet relaxation!

### 2.2 Banana Storage and Care
- **No Squishy-Squashy:** Store bananas in a cool place.
- **Sharing is Caring:** Ensure all fellow minions have a banana. We are a family!

---

## Chapter 3: **Gadget Safety and Handling**

### 3.1 Understanding Your Gadgets
- **Identifier Tags:** Each gadget has a unique tag. Always check the code before use.
- **Safety First!** Check condition: No cracks, no sparks, and definitely no mysterious ticking!

### 3.2 Common Gadgets Overview
- **Freeze Ray:** 
  - **Function:** Immobilizes targets; best for quick escapes.
  - **Safety:** Never aim at other minions; aim at walls or floors if uncertain.
- **Fart Gun:**
  - **Function:** Diversion tool; creates amusing distractions.
  - **Safety:** Not in closed areas. Fart responsibly!

### 3.3 Troubleshooting Gadgets
- **Technical Glitch:** Refer to gadget manual page 42 for resets.
- **Will Not Start:** Check battery life (most can be banana-activated too!)

### 3.4 Checklist Before Operation
- [ ] Ensure gadget is fully charged.
- [ ] Test the basic function in a safe area.
- [ ] Secure all loose clothing or accessories.

---

## Chapter 4: **Final Checkpoints**

- **Teamwork Build-y-up:** Help each other learn, work, and play. Collaborate for more efficiency and more fun.
- **Feedback Time:** Always learn from mistakes — like spilling banana pudding on gadgets!

### Conclusion
Remember, we are a minion family full of "BELLO!", bananas, and giggles! Follow protocols, respect gadgets, and spread minion joy everywhere!

[[Gru's Signature]]   
*Gru, Leader of the Minions*

---

Now, go forth and be the best minion ever! With lotsa lot of ba-ba-ba-blessed BANANAAA! 🎉

### Minion Worker Result (leadership-focused)

# Minion Leadership Manual: Rise to the Top!

## Introduction

Congratulations, future leader minion! Welcome to Gru's esteemed cadre of innovative thinkers and doers. This manual will guide you through the essentials of leading our squishy yellow crew with precision and pizzazz. Remember, with great banana comes great responsibility!  

---

## Chapter 1: The Strategic Significance of Banana Breaks

### The Power of the Banana
- **Fuel & Fun**: Bananas give minions energy—super strong, super fast!
- **Team Spirit**: Shared banana breaks boost camaraderie. Chat "BANANA!" to bond. 

### Leadership Insight:
- **Timing is Key**: Plan breaks effectively. Keep minions happy without impacting evil plans. 

### Leadership Scenario:
- You have a big evil mission upcoming. When's the best time to schedule a banana break to maximize efficiency and focus?
  - A) Right before the mission to energize the team.
  - B) Midway through for a power recharge.
  - C) After completion to celebrate success.

*Correct Answer: A or B - Timing depends on mission demands. Be adaptable!*

---

## Chapter 2: Advanced Gadget Management

### Gadget Know-How
- **Safety First!**: Protect those goggles and gloves, minion!
- **Buddy System**: Test gadgets with a partner—two goggles are better than one!

### Leader’s Role:
- **Assign Right Fits**: Choose minions for gadgets based on skills and experience!
- **Innovate & Learn**: Encourage experimentation but with caution, always Zzzz careful! 💤

### Leadership Quiz:
- Which statement reflects good gadget management?
  1. Letting anyone try a gadget at any time.
  2. Only allowing experienced minions to handle experimental gadgets.
  3. Never using gadgets, just in case.

*Correct Answer: 2 - Responsible gadget management enhances team safety!*

---

## Chapter 3: "BELLO!" Etiquette

### Communicate Like a Leader!
- **First Impressions Matter**: A confident "BELLO!" sets the tone. Practice vocal warm-ups.
- **Body Language**: Combine with big gestures. A good leader engages with enthusiasm!

### Leadership Tips:
- **Active Listening**: Respond to your fellow minions' "BELLO!"s. Shows respect. 
- **Encourage Inclusiveness**: Ensure everyone feels brave enough to speak up.

### Leadership Scenario:
- A minion is nervous about speaking up. How do you lead by example?
  - A) Give them a thumbs-up and a warm "BELLO!" 
  - B) Wait for them to speak first, then nod.
  - C) Ignore them as you are busy.

*Correct Answer: A - Positive reinforcement fosters confident communication!*

---

## Conclusion

Leadership is about inspiring your yellow brethren with clarity, creativity, and an unlimited affection for bananas. With the skills in this manual, you're set to elevate operations and morale. Embrace your role with zest, and never forget your inner "BANANA!" joy!

## Join the Bananas Quiz!

- Which element is crucial in leadership communication?
  1. Loudness
  2. Clarity
  3. Flourishing hand gestures

*Correct Answer: 2 and 3 - Clarity in message and gestures lead to effective communication!*

---

Minion Leader Motto: "BEE DOO BEE DOO!" Lead with light, lead with laughter. Go forth and be bananas! 🍌